## Inner Workings of RNNs

Now that we have a basic understanding and a bird's-eye view of how RNNs (Recurrent Neural Networks) work, let's explore some basic computations that the RNN cells have to do to produce the hidden states and outputs.

In the first step, a hidden state will usually be seeded as a matrix of zeros, so that it can be fed into the RNN cell together with the first input in the sequence. In the simplest RNNs, the hidden state and the input data will be multiplied with weight matrices initialized via a scheme such as Xavier or Kaiming (you can read more on this topic [here](link-to-more-info)). The result of these multiplications will then be passed through an activation function (such as a tanh function) to introduce non-linearity.

$$
hidden_t = \tanh(weight_{hidden} \cdot hidden_{t-1} + weight_{input} \cdot input_t)
$$

Additionally, if we require an output at the end of each time step, we can pass the hidden state that we just produced through a linear layer or just multiply it by another weight matrix to obtain the desired shape of the result.

$$
output_t = weight_{output} \cdot hidden_t
$$

The hidden state that we just produced will then be fed back into the RNN cell together with the next input, and this process continues until we run out of input or the model is programmed to stop producing outputs.

As mentioned earlier, these computations presented above are just simple representations of how RNN cells do their calculations. For the more advanced RNN structures such as LSTMs, GRUs, etc., the computations are generally much more complicated.


In the Markdown (MDX) format, I've used backticks to represent the mathematical equations inline. Please replace `link-to-more-info` with the appropriate link to read more about Xavier and Kaiming initialization if you have one.

In [179]:
import numpy as np
class DummyRNN():

  def __init__(self, input_features, output_features):
    self.Whi = np.random.random((output_features, output_features))
    self.Wxi = np.random.random((output_features, input_features))
    self.Who = np.random.random((output_features, output_features))
    self.b1 = np.random.random((output_features,))
    self.b2 = np.random.random((output_features,))
    self.init_hidden = np.zeros(output_features)

  def forward(self, input_data):
    hidden_in = self.init_hidden
    memory = []  
    for input_t in input_data:
      hidden_activation_t =  hidden_in @ self.Whi+ self.Wxi @ input_t + self.b1
      print(hidden_activation_t.shape)
      output = np.tanh(self.Who @ hidden_activation_t + self.b2)
      memory.append(output)
      hidden_in = hidden_activation_t
    return np.array(memory)

time_steps = 10
input_features = 1
output_features = 10

model = DummyRNN(input_features, output_features)
input_data = np.random.random((time_steps, input_features))
print(f"Input Shape: {input_data.shape}")
result = model.forward(input_data)
print(f"Output Shape: {result.shape}")
# Output shape: (output_features, time_steps)
result

Input Shape: (10, 1)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
Output Shape: (10, 10)


array([[0.99627527, 0.99943913, 0.99878452, 0.99980872, 0.99975101,
        0.99669862, 0.99972958, 0.99994519, 0.99989894, 0.99967911],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.       

In [3]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, \
TensorDataset
from torch.nn.utils import rnn as rnn_utils


In [252]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Your dataset
# Create the sequence data
X = np.arange(1, 46).reshape(-1, 3,1)
Y = np.array([y.sum() for y in X])

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
X_tensor

tensor([[[ 1.],
         [ 2.],
         [ 3.]],

        [[ 4.],
         [ 5.],
         [ 6.]],

        [[ 7.],
         [ 8.],
         [ 9.]],

        [[10.],
         [11.],
         [12.]],

        [[13.],
         [14.],
         [15.]],

        [[16.],
         [17.],
         [18.]],

        [[19.],
         [20.],
         [21.]],

        [[22.],
         [23.],
         [24.]],

        [[25.],
         [26.],
         [27.]],

        [[28.],
         [29.],
         [30.]],

        [[31.],
         [32.],
         [33.]],

        [[34.],
         [35.],
         [36.]],

        [[37.],
         [38.],
         [39.]],

        [[40.],
         [41.],
         [42.]],

        [[43.],
         [44.],
         [45.]]])

In [23]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm 
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, seq_len=5, max_len=1000):
        # Initialize the base class (Dataset)
        super(CustomDataset, self).__init__()
        
        # Create a list of numbers from 0 to max_len
        self.datalist = np.arange(0, max_len)
        
        # Split the data into sequences of length seq_len and their corresponding targets
        self.data, self.targets = self.timeseries(self.datalist, seq_len)
        
    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.data)
    
    def timeseries(self, data, window):
        # Create sequences and corresponding targets from the data
        temp = []
        targ = data[window:]
        for i in range(len(data) - window):
            temp.append(data[i:i + window])

        return np.array(temp), targ
    
    def __getitem__(self, index):
        # Get a single sample from the dataset
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x, y

# Create an instance of the CustomDataset with specified parameters
dataset = CustomDataset(seq_len=5, max_len=1000)
for x,y in dataset:
    print(x,y)
    break   
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


tensor([0., 1., 2., 3., 4.]) tensor(5.)


In [32]:
np.random.randn(2,3)

array([[-1.48960531,  1.17588816,  0.64131081],
       [-0.19888345,  0.79987576, -1.24990254]])

In [54]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm 
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import seaborn as sns
class CustomDataset(Dataset):
    def __init__(self,input_seq = 5,sample  = 100):
        super(CustomDataset).__init__()
        self.data = np.arange(1, (input_seq*sample)+1).reshape(-1, input_seq, 1)
        self.targets = self.sum(self.data)
        
    def __len__(self):
        return len(self.data)
    
    def sum(self, data):
        Y = np.array([y.sum() for y in data])
        return  Y 
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x,y
    
dataset = CustomDataset(input_seq = 3,sample  = 100)
for x,y in dataset:
    print(x,y)
    break
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


tensor([[1.],
        [2.],
        [3.]]) tensor(6.)


In [55]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # hidden states not defnined hence the value of h0,c0 == (0,0)
        out, (hn, cn) = self.rnn(x)
        # OUT: B,L,F
        
        # as the diagram suggest to take the last output in many to one 

        # all batch, last seq, all hidden values
        out = out[:, -1, :]
        # OUT: B,L[last],F ->final output of last time step
        out = self.fc(out)
        
        return out
    
    
model = RNN(input_size=1, hidden_size=256, num_layers=2)
t = torch.tensor([11,12,13]).type(torch.Tensor).view(1,-1,1)
print(f"Input: {t.shape}")
model(t)

Input: torch.Size([1, 3, 1])


tensor([[0.0176]], grad_fn=<AddmmBackward0>)

In [56]:
loss_function = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for e in tqdm(range(1000)):
    i = 0
    avg_loss = []
    for x,y in dataloader:
        optimizer.zero_grad()
        # forward
        predictions = model(x)
        
        # loss
        loss = loss_function(predictions, y)
        
        # backward
        loss.backward()

        # optimization
        optimizer.step()
        avg_loss.append(loss.detach().numpy())

        i+=1
    if e%5==0:
        avg_loss = np.array(avg_loss)
        print(avg_loss.mean())

  0%|          | 1/1000 [00:00<14:06,  1.18it/s]

265941.56


  1%|          | 6/1000 [00:03<10:00,  1.66it/s]

234120.02


  1%|          | 11/1000 [00:06<09:31,  1.73it/s]

...

 99%|█████████▉| 991/1000 [09:59<00:05,  1.79it/s]

63222.99


100%|█████████▉| 996/1000 [10:02<00:02,  1.76it/s]

64033.81


100%|██████████| 1000/1000 [10:04<00:00,  1.65it/s]


In [59]:
input_tensor = torch.tensor([1,11,12]).type(torch.Tensor).view(1,-1,1)
model(input_tensor)

tensor([[338.7949]], grad_fn=<AddmmBackward0>)

In [3]:
import torch
import torch.nn as nn

N = 100
D = 3
hidden_unit = 50

X = torch.randn(N,D)
W = torch.randn(D,hidden_unit)
WX = torch.matmul(X,W)
print(WX.shape)
linear_layer = nn.Linear(in_features=D,out_features=hidden_unit)
wx = linear_layer(X)
print(wx.shape)

H = torch.randn(N,hidden_unit)
U = torch.randn(hidden_unit,hidden_unit)
HU = torch.matmul(H,U)
print(HU.shape)
linear_layer = nn.Linear(in_features=hidden_unit,out_features=hidden_unit)
hu = linear_layer(H)
print(hu.shape)

print((WX+HU).shape)

torch.Size([100, 50])
torch.Size([100, 50])
torch.Size([100, 50])
torch.Size([100, 50])
torch.Size([100, 50])


In [4]:
WX+HU + torch.randn(1,hidden_unit)

tensor([[-7.7264, -0.2913, -1.9945,  ...,  6.0025, -2.9757,  9.1364],
        [-2.2795, -4.2909,  6.3641,  ...,  7.9033, -3.2580, 16.0137],
        [12.5918,  0.8077,  2.4117,  ...,  0.7654, -8.3708,  1.3911],
        ...,
        [ 4.6413, -4.3244,  0.4369,  ...,  5.2540, -7.5529, -3.1038],
        [10.0056,  0.2125, -5.2950,  ..., -2.8770,  3.0147, -8.1516],
        [ 9.3609,  2.3222,  7.0069,  ...,  1.0851,  1.7609,  0.2755]])

In [519]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNCell(nn.Module):
    def __init__(self, in_features, hidden_unit, output_unit):
        super(RNNCell, self).__init__()
        self.hidden_unit = hidden_unit

        # Define the parameters as PyTorch Parameters
        self.Wax = nn.Parameter(torch.randn(in_features, hidden_unit))
        self.Waa = nn.Parameter(torch.randn(hidden_unit, hidden_unit))
        self.Wya = nn.Parameter(torch.randn(hidden_unit, output_unit))
        self.ba = nn.Parameter(torch.zeros(1, hidden_unit))  
        self.by = nn.Parameter(torch.zeros(1, output_unit))  

    def forward(self, X, H):
        # Compute next activation state
        a_next = torch.tanh(torch.matmul(X, self.Wax) + torch.matmul(H, self.Waa) + self.ba)
        # print(a_next.shape)
        # Compute output of the current cell
        yt_pred = F.softmax(torch.matmul(a_next, self.Wya) + self.by, dim=1)  # Corrected dim=0 to dim=1

        return a_next, yt_pred


# Create an instance of the RNNCell module
N = 100
D = 1
hidden_unit = 200
rnn_cell = RNNCell(in_features=D , hidden_unit=hidden_unit, output_unit=2)

# Define the input data and previous hidden state as PyTorch tensors
Xt = torch.randn(N, D)
a_prev = torch.randn(N, hidden_unit)

# Perform forward pass through the RNNCell
a_next, yt_pred = rnn_cell(Xt, a_prev)

print("Next hidden state (a_next):")
print(a_next.shape)
print("Predicted output (yt_pred):")
print(yt_pred.shape)
print(Xt.shape)

Next hidden state (a_next):
torch.Size([100, 200])
Predicted output (yt_pred):
torch.Size([100, 2])
torch.Size([100, 1])


In [638]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
torch.random.manual_seed(123)
class CustomDataset(Dataset):
    def __init__(self, input_seq=5, sample=100):
        super(CustomDataset, self).__init__()
        self.data = np.arange(1, (input_seq * sample) + 1).reshape(-1, input_seq, 1)
        self.targets = self.sum(self.data)

    def __len__(self):
        return len(self.data)

    def sum(self, data):
        Y = np.array([y.sum() for y in data])
        return Y

    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x, y

# Create the CustomDataset and DataLoader
dataset = CustomDataset(input_seq=3, sample=10000)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define the RNNCell module
class RNNCell(nn.Module):
    def __init__(self, in_features, hidden_unit):
        super(RNNCell, self).__init__()
        self.hidden_unit = hidden_unit

        # Define the parameters as PyTorch Parameters
        self.Wax = nn.Parameter(torch.randn(in_features, hidden_unit))
        self.Waa = nn.Parameter(torch.randn(hidden_unit, hidden_unit))
        self.ba = nn.Parameter(torch.zeros(1, hidden_unit))
        self.fc = nn.Linear(hidden_unit, 1)  # Fully connected layer for Many-to-One

    def forward(self, X):
        # Get the batch size and sequence length from the input tensor
        batch_size, seq_length, _ = X.size()

        # Initialize the hidden state as zeros
        a_prev = torch.zeros(batch_size, self.hidden_unit)

        # List to store the output at each time step
        outputs = []

        # Loop through the sequence
        for t in range(seq_length):
            # Extract the current time step's input
            Xt = X[:, t, :]

            # Compute the next hidden state and predicted output
            a_next = torch.tanh(torch.matmul(Xt, self.Wax) + torch.matmul(a_prev, self.Waa) + self.ba)
            # Append the output to the list
            outputs.append(a_next)
            a_prev =  a_next

        last_hidden_state = outputs[-1]  # Take the final hidden state
        # print(last_hidden_state.shape) # 1,HU
        # Pass the final hidden state through the fully connected layer
        out = self.fc(last_hidden_state)
        return out 

model = RNNCell(in_features=1, hidden_unit=300)
input_tensor = torch.tensor([1,2,3,4,5]).type(torch.Tensor).view(1,-1,1)
model(input_tensor)


tensor([[-0.0731]], grad_fn=<AddmmBackward0>)

In [639]:
loss_function = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for e in tqdm(range(100)):
    for x,y in dataloader:
        optimizer.zero_grad()


        predictions = model(x)

        loss = loss_function(predictions.view(-1), y) # [N,1] -> [N]
        
        # backward
        loss.backward()

        # optimization
        optimizer.step()


100%|██████████| 100/100 [14:46<00:00,  8.86s/it]


In [640]:
input_tensor = torch.tensor([1,2,3,4,5]).type(torch.Tensor).view(1,-1,1)
model(input_tensor)

tensor([[257.0251]], grad_fn=<AddmmBackward0>)